# Nissen et al., 2023: Severe 21st-century OA in Antarctic MPAs
#
# script to save regional averages of a variety of carbonate chemistry variables as netcdf files (e.g., undersaturated volume below XXm)
# Antarctic Peninsula only!
# requires full model output
# note that some paths are hard-coded throughout the script

In [ ]:
#!jupyter nbconvert --to script save_carbonate_chemistry_regional.ipynb

In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
from matplotlib import cm
from netCDF4 import Dataset, MFDataset
import pandas as pd

In [2]:
#---
# load mesh info
#---

path_mesh = '/pscratch/sd/c/cnissen/'
file_mesh = 'Nissen2022_FESOM_REcoM_mesh_information_corrected_20220910.nc'

f1 = Dataset(path_mesh+file_mesh) #xr.open_dataset(path+file1)
lat      = f1.variables['lat'][:]
lon      = f1.variables['lon'][:]
zlevs    = f1.variables['zlevs'][:]
cavities = f1.variables['cavity'][:]
topo     = f1.variables['topo'][:]
area     = f1.variables['cell_area'][:]
volume   = f1.variables['cell_volume'][:]
f1.close()
print(lat.shape)


(72411,)


In [3]:
#-----
# load model output, calculate carbonate chemistry at each location, store in netcdf file
#-----
# what fields from model do I need?
#    tos, sos, alk (surface only), dic (surface only), sio2 (surface only), din (surface only), slp
#

#----
# some settings
#----

#savepath1    = '/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/carbonate_chemistry_3D/'
#if not os.path.exists(savepath1):
#    print('Created '+savepath1)
#    os.makedirs(savepath1)


In [4]:

def get_runID(which_sim,year):
    #------
    # get runID for simulation & year
    #------
    
    # simAssp585
    runID1 = 'COARZE_AWICM_wREcoM_hist_incrCO2_2'   # until 1991
    runID2 = 'COARZE_AWICM_wREcoM_hist_incrCO2_5'   # 1992-2014
    runID3 = 'COARZE_AWICM_wREcoM_ssp585_incrCO2_1' # after 2014

    # simB (CTRL)
    runID4 = 'COARZE_AWICM_wREcoM_constClim_constCO2_1' # until 2014
    runID5 = 'COARZE_AWICM_wREcoM_constClim_constCO2_2' # after 2014

    # simCssp585
    runID6 = 'COARZE_AWICM_wREcoM_constClim_varCO2_1' 

    # simD
    runID7 = 'COARZE_AWICM_wREcoM_varClim_constCO2_1'  # until 2014
    runID8 = 'COARZE_AWICM_wREcoM_varClim_constCO2_2'  # after 2014

    # simAssp245
    runID9 = 'COARZE_AWICM_wREcoM_ssp245_incrCO2_1' 

    # simCssp245
    runID10 = 'COARZE_AWICM_wREcoM_constClim_varCO2_ssp245_1' # after 2014; runID6 before

    if which_sim in ['simAssp585']:
        if year<1992:
            runID=runID1
        elif year>2014: # ssp585 scenario
            runID=runID3
        else:
            runID=runID2
    elif which_sim in ['simB']:
        if year>2014:
            runID = runID5
        else:
            runID = runID4
    elif which_sim in ['simAssp370']:
        if year<1992:
            runID = runID1
        elif year>2014:
            if year>2080:
                runID = runID6b
            else:
                runID = runID6a
        else:
            runID = runID2
    elif which_sim in ['simAssp245']:
        if year<1992:
            runID = runID1
        elif year>2014:
            runID = runID7
        else:
            runID = runID2
    elif which_sim in ['simAssp126']:
        if year<1992:
            runID = runID1
        elif year>2014:
            runID = runID8
        else:
            runID = runID2
            
    return runID


In [5]:
print (volume.shape)

(72411, 99)


In [6]:
#-----
# load volume
# convert to 3D field
#-----

   
depths2000 = np.unique(zlevs) #np.unique(mesh.zlev)
ind2000    = np.where(depths2000<5500)[0] 
depths2000 = depths2000[depths2000<5500]
num_depths = len(depths2000)
print ('num_depths:',num_depths)

depths = np.unique(zlevs)

volume_2d = np.transpose(volume)
print (volume.shape)
print (volume_2d.shape)

# restrict to top XXm or similar
volume_2d = volume_2d[ind2000,:]
print (volume_2d.shape)
print (np.nanmin(volume_2d),np.nanmax(volume_2d))



num_depths: 97
(72411, 99)
(99, 72411)
(97, 72411)
4698317.0 28607110000000.0


In [7]:
#----
# FUNCTIONS
#----

def get_intersection(lst1, lst2):  
        # Use of hybrid method 
        temp = set(lst2) 
        lst3 = [value for value in lst1 if value in temp] 
        return lst3
    
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

# get subareas based on lat only
def get_subarea_indices_sectors_lat_only(lat,lon):
    #subareas  = ['Larsen/FRIS','eWeddell','Amery/Australian','Ross Sea','Amundsen/Bellingh']
    
    lat_threshold = -60 #-60 # 0=all SH data
        
    ross1 = 150 #160
    ross2 = -130 # -145
    
    eWeddell1 = -10
    
    print ('Subareas: south of '+str(-1*lat_threshold)+'°S...')
        
    shelf_ind = np.zeros(len(lon))
    for i in range(1,len(lon)+1):
        dd = lon[i-1]
        if lat[i-1]<lat_threshold and dd>-52 and dd<eWeddell1: # Filchner-Ronne & Larsen 
            shelf_ind[i-1]=1
        elif lat[i-1]<-74 and dd>-85 and dd<eWeddell1: # Filchner-Ronne & Larsen in the very south
            shelf_ind[i-1]=1
        elif lat[i-1]<lat_threshold and dd>eWeddell1 and dd<30: # eWeddell
            shelf_ind[i-1]=6
        elif lat[i-1]<lat_threshold and dd>30 and dd<90: # Prydz
            shelf_ind[i-1]=2
        elif lat[i-1]<lat_threshold and dd>90 and dd<ross1: # Amery & Australian
            shelf_ind[i-1]=3
        elif lat[i-1]<lat_threshold and dd>ross1:  # Ross Sea 
            shelf_ind[i-1]=4
        elif lat[i-1]<lat_threshold and dd<ross2:   # Ross Sea
            shelf_ind[i-1]=4
        elif lat[i-1]<lat_threshold and dd>ross2 and dd<-69: #dd<-66: # Amundsen & Bellinghausen
            shelf_ind[i-1]=5
        elif lat[i-1]<lat_threshold and dd>-69 and dd<-52: #dd<-66: # Antarctic Peninsula
            wap_lat = -70
            if lat[i-1]>wap_lat:
                shelf_ind[i-1]=7 # pensinsula
            else:
                #shelf_ind[i-1]=7 
                # now find correct separation between Amundsen & Weddell Sea
                if dd>-66:
                    shelf_ind[i-1]=1
                else:
                    shelf_ind[i-1]=5
                    
            
        # now find correct separation between Amundsen & Weddell Sea
        #if lat[i-1]<=wap_lat:
            
                
        
 #       # now find correct separation between Amundsen & Weddell Sea
 #       if lat[i-1]<-66 and dd>-63 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-66 and lat[i-1]>-70 and dd>-64.5 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-66.8 and lat[i-1]>-70 and dd>-66 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64 and lat[i-1]>-66 and dd>-60.6 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64.8 and lat[i-1]>-66 and dd>-62.4 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64.7 and lat[i-1]>-65.9 and dd>-62.2 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-63.4 and lat[i-1]>-64.5 and dd>-58 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
            
    return shelf_ind

In [8]:
#---
# plot map of subregions
#----

shelf_ind = get_subarea_indices_sectors_lat_only(lat,lon)

ind = np.where((topo>2000) & (cavities==0))[0]
shelf_ind[ind]=0
shelf_ind[cavities==1]=0

weddell = np.zeros_like(shelf_ind)
weddell[shelf_ind==1]=1

eWeddell = np.zeros_like(shelf_ind)
eWeddell[shelf_ind==6]=1

prydz = np.zeros_like(shelf_ind)
prydz[shelf_ind==2]=1

adelie = np.zeros_like(shelf_ind)
adelie[shelf_ind==3]=1

ross = np.zeros_like(shelf_ind)
ross[shelf_ind==4]=1

amundsen = np.zeros_like(shelf_ind)
amundsen[shelf_ind==5]=1

wap = np.zeros_like(shelf_ind)
wap[shelf_ind==7]=1

#----
# get masks for MPAs
#----

path1 = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/masks_MPAs/'

f1 = Dataset(path1+'MPA_mask_AntarcticPeninsula_fesom_mesh_COARZE.nc')
mask_WAP_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_1_fesom_mesh_COARZE.nc')
mask_eastAA1_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_2_fesom_mesh_COARZE.nc')
mask_eastAA2_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_3_fesom_mesh_COARZE.nc')
mask_eastAA3_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_Orkney_fesom_mesh_COARZE.nc')
mask_Orkney_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_RossSea_fesom_mesh_COARZE.nc')
mask_RossSea_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_WeddellSea_fesom_mesh_COARZE.nc')
mask_WeddellSea_MPA = f1.variables['mask_fesom'][:]
f1.close()

#------------

subareas  = ['Larsen/FRIS','eWeddell','Amery/Australian','Ross Sea','Amundsen/Bellingh',\
             'south of SR4','south of 60S']
subareas = ['south of SR4']
# ADD "south_of_SR4" and "south_of_60S"
# additionally, load masks for "south of SR4" and "south of 60
path_mask_SR4 = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_SR4.nc'
path_mask_60S = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_60S.nc'
path_mask_shelfBreak = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_shelf_break.nc'
fileSR4 = Dataset(path_mask_SR4)
file60S = Dataset(path_mask_60S)
file_shelfBreak = Dataset(path_mask_shelfBreak)
maskSR4 = fileSR4.variables['mask'][:]
mask60S = file60S.variables['mask'][:]
mask_shelfBreak = file_shelfBreak.variables['mask'][:]

print (np.min(shelf_ind),np.max(shelf_ind))
latlim = -60
depth_threshold = 2000

#-----
# plot map of where subareas are
#-----

from matplotlib.colors import ListedColormap

# make the color map:
cmap1 = ListedColormap(['white','darkblue','cornflowerblue','firebrick',\
                        'gold','orange','mediumpurple','violet','mediumturquoise'])
cmap1 = ListedColormap(['white',(51./255.,34./255.,136./255.),(136./255.,204./255.,238./255.),\
                  (68./255.,170./255.,153./255.),(17./255.,119./255.,51./255.),\
                  (153./255.,153./255.,51./255.),(221./255.,204./255.,119./255.),\
                   (204./255.,102./255.,119./255.),(136./255.,34./255.,85./255.)])

levels1 = np.arange(0,10,1)

plot_regions = False
if plot_regions:
    dpicnt = 200
    elem_no_nan = mesh.elem[mesh.no_cyclic_elem,:]

    fig8= plt.figure(num=18, figsize=(4,3), dpi=200, facecolor='w', edgecolor='k')
    map = Basemap(projection='spstere',boundinglat=-60,lon_0=-180,resolution='l')
    #map.drawcoastlines()
    map.drawcountries()#
    map.drawmapboundary(fill_color='0.7')
    x, y = map(mesh.x2, mesh.y2)
    shelf_ind[shelf_ind==0]=-999
    mm=plt.tricontourf(x, y, elem_no_nan[::], shelf_ind, levels = levels1, \
                        cmap=cmap1, extend='both') #,vmin=-4, vmax=4)
    cbar = plt.colorbar(orientation='horizontal',fraction=0.075, pad=0.03,shrink=0.75)# ,ticks=cticks)
    #cbar.cmap.set_under('white')
    cbar.remove() 
    #plt.draw()
    lw = 1.2

    topo_plot = np.copy(mesh.topo)
    topo_plot[cavities==1]=-999
    m2=plt.tricontour(x, y, elem_no_nan[::], topo_plot, levels = [2000],linewidths=[lw],colors=['black'])

    save_plot = False
    if save_plot:
        plt.savefig(savepath1+'Map_subareas_for_C_anth_analysis.png',\
                                            dpi = dpicnt, bbox_inches='tight') 
    plt.show()
    #plt.close(fig8)

# Weddell
# eWeddell
# Prydz
# Adelie
# Ross
# Amundsen/Bellinghausen
# Antarctic Pensinsula


Subareas: south of 60°S...
0.0 7.0


In [9]:
#---
# subareas
#---

#weddell[shelf_ind==1]=1
#eWeddell[shelf_ind==6]=1
#prydz[shelf_ind==2]=1
#adelie[shelf_ind==3]=1
#ross[shelf_ind==4]=1
#amundsen[shelf_ind==5]=1
#wap[shelf_ind==7]=1

# for MPAs (outside cavities only for now):
# 1) complete MPA
# 2) shelf only
# 3) open ocean only

subregions = ['all_shelves','WeddellSea','eWeddellSea','Prydz','Adelie','RossSea','Amundsen_Bellingshausen','WAP',\
             'RossSea_MPA','RossSea_MPA_shelf','RossSea_MPA_openOcean',\
              'WeddellSea_MPA','WeddellSea_MPA_shelf','WeddellSea_MPA_openOcean',\
              'Orkney_MPA','AntarcticPeninsula_MPA',\
             'East_Antarctica_1_MPA','East_Antarctica_2_MPA','East_Antarctica_3_MPA','East_Antarctica_all_MPA',\
             'East_Antarctica_1_MPA_shelf','East_Antarctica_2_MPA_shelf',\
              'East_Antarctica_3_MPA_shelf','East_Antarctica_all_MPA_shelf',\
             'East_Antarctica_1_MPA_openOcean','East_Antarctica_2_MPA_openOcean',\
              'East_Antarctica_3_MPA_openOcean','East_Antarctica_all_MPA_openOcean']
#subregions = ['all_shelves']#,'WeddellSea','RossSea','Amundsen_Bellingshausen','WAP']


In [10]:
print(area.shape)

(72411,)


In [10]:
#----
# loop over simulations and years
#----
from tqdm import tqdm

#indSO = np.where((mesh.y2<-60.0) & (topo<2000.0) & (cavities==0))[0]
#print ('Number of nodes:',indSO.shape)

which_sim = 'simB'
path1 = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/'+which_sim+'/' 

if which_sim in ['simAssp126','simAssp245','simAssp370','simCssp245']:
    year_list1 = np.arange(2015,2100+1,1) 
elif which_sim in ['simAssp585','simB','simCssp585']:
    year_list1 = np.arange(1990,2100+1,1) 
    
print(year_list1)

# depth levels to avg over
ind1 = np.where(depths2000<=100)[0]
ind2 = np.where((depths2000>100) & (depths2000<=500))[0]
ind3 = np.where((depths2000>500))[0]

ind4 = np.where(depths2000<=10)[0]
ind5 = np.where(depths2000<=20)[0]
ind6 = np.where(depths2000<=50)[0]

ind7 = np.where((depths2000>0) & (depths2000<=10))[0]
ind8 = np.where((depths2000>10) & (depths2000<=100))[0]

ind9 = np.where((depths2000<=2000))[0]

ind10 = np.where(depths2000<=200)[0]

process_top100    = False
process_100_500   = False
process_below500  = False

process_top10     = False
process_top20     = False
process_top50     = False

process_0_10      = False
process_10_100    = False

process_above2000 = False
process_top200    = True

process_water_column = False
process_surface      = False

#---
# Make sure to define the correct variable list!!
# currently still hard-coded
#---
#vari_list = ['omegaA_avg_srf','omegaA_avg_top10','omegaA_avg_top20','omegaA_avg_top50',\
#                         'omegaA_avg_top100','omegaA_avg_100_500','omegaA_avg_below500','omegaA_avg_water_column',\
#                         'omegaA_avg_0_10','omegaA_avg_10_100','omegaA_avg_above2000',\
#                         #
#                        'omegaC_avg_srf','omegaC_avg_top10','omegaC_avg_top20','omegaC_avg_top50',\
#                         'omegaC_avg_top100','omegaC_avg_100_500','omegaC_avg_below500','omegaC_avg_water_column',\
#                         'omegaC_avg_0_10','omegaC_avg_10_100','omegaC_avg_above2000',\
#                         #
#                        'pH_avg_srf','pH_avg_top10','pH_avg_top20','pH_avg_top50',\
#                         'pH_avg_top100','pH_avg_100_500','pH_avg_below500','pH_avg_water_column',\
#                         'pH_avg_0_10','pH_avg_10_100','pH_avg_above2000',\
#                         #
#                        'omegaA_vol_below1_srf','omegaA_vol_below1_top10','omegaA_vol_below1_top20',\
#                         'omegaA_vol_below1_top50','omegaA_vol_below1_top100','omegaA_vol_below1_100_500',\
#                         'omegaA_vol_below1_below500','omegaA_vol_below1_water_column',\
#                         'omegaA_vol_below1_0_10','omegaA_vol_below1_10_100','omegaA_vol_below1_above2000',\
#                         #
#                        'omegaC_vol_below1_srf','omegaC_vol_below1_top10','omegaC_vol_below1_top20',\
#                         'omegaC_vol_below1_top50','omegaC_vol_below1_top100','omegaC_vol_below1_100_500',\
#                         'omegaC_vol_below1_below500','omegaC_vol_below1_water_column',\
#                         'omegaC_vol_below1_0_10','omegaC_vol_below1_10_100','omegaC_vol_below1_above2000',\
#                        #
#                        'omegaA_vol_below05_srf','omegaA_vol_below05_top10','omegaA_vol_below05_top20',\
#                         'omegaA_vol_below05_top50','omegaA_vol_below05_top100','omegaA_vol_below05_100_500',\
#                         'omegaA_vol_below05_below500','omegaA_vol_below05_water_column',\
#                         'omegaA_vol_below05_0_10','omegaA_vol_below05_10_100','omegaA_vol_below05_above2000',\
#                         #
#                        'omegaC_vol_below05_srf','omegaC_vol_below05_top10','omegaC_vol_below05_top20',\
#                         'omegaC_vol_below05_top50','omegaC_vol_below05_top100','omegaC_vol_below05_100_500',\
#                         'omegaC_vol_below05_below500','omegaC_vol_below05_water_column',\
#                         'omegaC_vol_below05_0_10','omegaC_vol_below05_10_100','omegaC_vol_below05_above2000',\
#                        #
#                        'omegaA_vol_below15_srf','omegaA_vol_below15_top10','omegaA_vol_below15_top20',\
#                         'omegaA_vol_below15_top50','omegaA_vol_below15_top100','omegaA_vol_below15_100_500',\
#                         'omegaA_vol_below15_below500','omegaA_vol_below15_water_column',\
#                         'omegaA_vol_below15_0_10','omegaA_vol_below15_10_100','omegaA_vol_below15_above2000',\
#                         #
#                        'omegaC_vol_below15_srf','omegaC_vol_below15_top10','omegaC_vol_below15_top20',\
#                         'omegaC_vol_below15_top50','omegaC_vol_below15_top100','omegaC_vol_below15_100_500',\
#                         'omegaC_vol_below15_below500','omegaC_vol_below15_water_column',\
#                         'omegaC_vol_below15_0_10','omegaC_vol_below15_10_100','omegaC_vol_below15_above2000',\
#                        #
#                        'omegaA_vol_above15_srf','omegaA_vol_above15_top10','omegaA_vol_above15_top20',\
#                         'omegaA_vol_above15_top50','omegaA_vol_above15_top100','omegaA_vol_above15_100_500',\
#                         'omegaA_vol_above15_below500','omegaA_vol_above15_water_column',\
#                         'omegaA_vol_above15_0_10','omegaA_vol_above15_10_100','omegaA_vol_above15_above2000',\
#                         #
#                        'omegaC_vol_above15_srf','omegaC_vol_above15_top10','omegaC_vol_above15_top20',\
#                         'omegaC_vol_above15_top50','omegaC_vol_above15_top100','omegaC_vol_above15_100_500',\
#                         'omegaC_vol_above15_below500','omegaC_vol_above15_water_column',\
#                        'omegaC_vol_above15_0_10','omegaC_vol_above15_10_100','omegaC_vol_above15_above2000',\
#                        #
#                        'omegaA_vol_above2_srf','omegaA_vol_above2_top10','omegaA_vol_above2_top20',\
#                         'omegaA_vol_above2_top50','omegaA_vol_above2_top100','omegaA_vol_above2_100_500',\
#                         'omegaA_vol_above2_below500','omegaA_vol_above2_water_column',\
#                         'omegaA_vol_above2_0_10','omegaA_vol_above2_10_100','omegaA_vol_above2_above2000',\
#                         #
#                        'omegaC_vol_above2_srf','omegaC_vol_above2_top10','omegaC_vol_above2_top20',\
#                         'omegaC_vol_above2_top50','omegaC_vol_above2_top100','omegaC_vol_above2_100_500',\
#                         'omegaC_vol_above2_below500','omegaC_vol_above2_water_column',\
#                        'omegaC_vol_above2_0_10','omegaC_vol_above2_10_100','omegaC_vol_above2_above2000']

#vari_list = ['omegaA_avg_above2000',\
#                         'omegaC_avg_above2000',\
#                         'pH_avg_above2000',\
#                         'omegaA_vol_below1_above2000',\
#                         'omegaC_vol_below1_above2000',\
#                        'omegaA_vol_below05_above2000',\
#                         'omegaC_vol_below05_above2000',\
#                        'omegaA_vol_below15_above2000',\
#                        'omegaC_vol_below15_above2000',\
#                        'omegaA_vol_above15_above2000',\
#                         'omegaC_vol_above15_above2000',\
#                        'omegaA_vol_above2_above2000',\
#                         'omegaC_vol_above2_above2000']

vari_list = ['omegaA_avg_top200',\
                         'omegaC_avg_top200',\
                         'pH_avg_top200',\
                         'omegaA_vol_below1_top200',\
                         'omegaC_vol_below1_top200',\
                        'omegaA_vol_below05_top200',\
                         'omegaC_vol_below05_top200',\
                        'omegaA_vol_below15_top200',\
                        'omegaC_vol_below15_top200',\
                        'omegaA_vol_above15_top200',\
                         'omegaC_vol_above15_top200',\
                        'omegaA_vol_above2_top200',\
                         'omegaC_vol_above2_top200']


if process_water_column:
    # full water column
    omegaA_avg_shelf1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst1_gt2 = np.zeros([12,len(year_list1),len(subregions)])

if process_surface:
    # surface only
    omegaA_avg_shelf_srf1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_srf1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_srf1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_srf1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_srf1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_srf1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_srf1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_srf1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_srf1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_srf1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_srf1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_srf1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_srf1_gt2 = np.zeros([12,len(year_list1),len(subregions)])

if process_top10:
    # top10 m
    omegaA_avg_shelf_top10_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_top10_1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_top10_1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top10_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top10_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top10_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top10_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top10_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top10_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top10_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top10_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top10_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top10_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])

if process_top20:
    # top20 m
    omegaA_avg_shelf_top20_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_top20_1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_top20_1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top20_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top20_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top20_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top20_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top20_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top20_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top20_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top20_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top20_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top20_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    
if process_top50:
    # top50 m
    omegaA_avg_shelf_top50_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_top50_1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_top50_1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top50_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top50_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top50_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top50_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top50_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top50_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top50_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top50_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top50_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top50_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    
if process_top100: 
    # top100 m
    omegaA_avg_shelf_top100_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_top100_1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_top100_1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top100_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top100_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top100_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top100_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top100_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top100_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top100_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top100_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top100_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top100_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    
if process_top200: 
    # top100 m
    omegaA_avg_shelf_top200_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_top200_1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_top200_1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top200_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top200_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top200_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top200_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top200_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top200_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top200_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top200_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_top200_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_top200_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    
if process_0_10:
    # 0-10m
    omegaA_avg_shelf_0_10_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_0_10_1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_0_10_1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_0_10_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_0_10_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_0_10_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_0_10_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_0_10_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_0_10_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_0_10_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_0_10_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_0_10_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_0_10_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])

if process_10_100:
    # 10-100m
    omegaA_avg_shelf_10_100_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_10_100_1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_10_100_1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_10_100_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_10_100_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_10_100_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_10_100_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_10_100_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_10_100_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_10_100_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_10_100_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_10_100_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_10_100_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])

if process_100_500:
    # 100-500m
    omegaA_avg_shelf_100_500_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_100_500_1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_100_500_1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_100_500_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_100_500_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_100_500_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_100_500_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_100_500_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_100_500_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_100_500_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_100_500_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_100_500_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_100_500_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])

if process_below500:
    # below 500m
    omegaA_avg_shelf_below500_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_below500_1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_below500_1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_below500_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_below500_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_below500_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_below500_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_below500_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_below500_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_below500_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_below500_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_below500_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_below500_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])

if process_above2000:
    # above 2000m 
    omegaA_avg_shelf_above2000_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_avg_shelf_above2000_1 = np.zeros([12,len(year_list1),len(subregions)])
    pH_avg_shelf_above2000_1     = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_above2000_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_above2000_1 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_above2000_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_above2000_1_05 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_above2000_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_above2000_1_15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_above2000_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_above2000_1_gt15 = np.zeros([12,len(year_list1),len(subregions)])
    omegaA_vol_underst_above2000_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])
    omegaC_vol_underst_above2000_1_gt2 = np.zeros([12,len(year_list1),len(subregions)])

# loop over years
for yy in range(0,28):#28,len(year_list1)):
    print('year '+str(year_list1[yy])+'...')
                
    #runID = get_runID(which_sim,year_list[yy])

    # open netcdf files
    f1 = Dataset(path1+'OmegaA_fesom_'+str(year_list1[yy])+'0101.nc')
    f2 = Dataset(path1+'OmegaC_fesom_'+str(year_list1[yy])+'0101.nc')
    f3 = Dataset(path1+'pH_fesom_'+str(year_list1[yy])+'0101.nc') 
    
    for mm in tqdm(range(0,12)):
        #print ('Month '+str(mm+1)+'...')
        
        for rr in range(0,len(subregions)):
            #print subregions[rr]
            if subregions[rr] in ['all_shelves']:
                ind_regions = np.where(shelf_ind>0)[0]
            elif subregions[rr] in ['WeddellSea']:
                ind_regions = np.where(weddell==1)[0]
            elif subregions[rr] in ['eWeddellSea']:
                ind_regions = np.where(eWeddell==1)[0]
            elif subregions[rr] in ['Prydz']:
                ind_regions = np.where(prydz==1)[0]
            elif subregions[rr] in ['Adelie']:
                ind_regions = np.where(adelie==1)[0]
            elif subregions[rr] in ['RossSea']:
                ind_regions = np.where(ross==1)[0]
            elif subregions[rr] in ['Amundsen_Bellingshausen']:
                ind_regions = np.where(amundsen==1)[0]
            elif subregions[rr] in ['WAP']:
                ind_regions = np.where(wap==1)[0]
            
            elif subregions[rr] in ['RossSea_MPA']:
                ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0))[0]
            elif subregions[rr] in ['RossSea_MPA_shelf']:
                ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['RossSea_MPA_openOcean']:
                ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0) & (topo>2000))[0]
            elif subregions[rr] in ['WeddellSea_MPA']:
                ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0))[0]
            elif subregions[rr] in ['WeddellSea_MPA_shelf']:
                ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['WeddellSea_MPA_openOcean']:
                ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0) & (topo>2000))[0]
            elif subregions[rr] in ['AntarcticPeninsula_MPA']:
                ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0))[0]
            elif subregions[rr] in ['AntarcticPeninsula_MPA_shelf']:
                ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['AntarcticPeninsula_openOcean']:
                ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0) & (topo>2000))[0]
            elif subregions[rr] in ['Orkney_MPA']:
                ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0))[0]
            elif subregions[rr] in ['Orkney_MPA_shelf']:
                ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['Orkney_MPA_openOcean']:
                ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0) & (topo>2000))[0]
            
            
            elif subregions[rr] in ['East_Antarctica_1_MPA']:
                ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0))[0]
            elif subregions[rr] in ['East_Antarctica_1_MPA_shelf']:
                ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['East_Antarctica_1_MPA_openOcean']:
                ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0) & (topo>2000))[0]
            elif subregions[rr] in ['East_Antarctica_2_MPA']:
                ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0))[0]
            elif subregions[rr] in ['East_Antarctica_2_MPA_shelf']:
                ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['East_Antarctica_2_MPA_openOcean']:
                ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0) & (topo>2000))[0]
            elif subregions[rr] in ['East_Antarctica_3_MPA']:
                ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0))[0]
            elif subregions[rr] in ['East_Antarctica_3_MPA_shelf']:
                ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['East_Antarctica_3_MPA_openOcean']:
                ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0) & (topo>2000))[0]
            
            elif subregions[rr] in ['East_Antarctica_all_MPA']:
                ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0))[0]
            elif subregions[rr] in ['East_Antarctica_all_MPA_shelf']:
                ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['East_Antarctica_all_MPA_openOcean']:
                ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0) & (topo>2000))[0]

            if process_top10:
                #----
                # top10m
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[ind4,:]
                omegaC = omegaC[ind4,:]
                pH = pH[ind4,:]
                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                #ind_av = omegaA.mask==False
                ind_av = np.where(omegaA>0)[0]
                vv = volume_2d[ind4,:]
                vol = vv[:,ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_top10_1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_top10_1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_top10_1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_top10_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_top10_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_top10_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_top10_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_top10_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_top10_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_top10_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_top10_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_top10_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_top10_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low


                del ind_av,vol,omegaA,omegaC,pH
            
            if process_top20:
                #----
                # top20m
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[ind5,:]
                omegaC = omegaC[ind5,:]
                pH = pH[ind5,:]
                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                #ind_av = omegaA.mask==False
                ind_av = np.where(omegaA>0)[0]
                vv = volume_2d[ind5,:]
                vol = vv[:,ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_top20_1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_top20_1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_top20_1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_top20_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_top20_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_top20_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_top20_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_top20_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_top20_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_top20_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_top20_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_top20_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_top20_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                del ind_av,vol,omegaA,omegaC,pH

            if process_top50:
                #----
                # top50m
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[ind6,:]
                omegaC = omegaC[ind6,:]
                pH = pH[ind6,:]
                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                #ind_av = omegaA.mask==False
                ind_av = np.where(omegaA>0)[0]
                vv = volume_2d[ind6,:]
                vol = vv[:,ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_top50_1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_top50_1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_top50_1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_top50_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_top50_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_top50_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_top50_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_top50_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_top50_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_top50_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_top50_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_top50_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_top50_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                del ind_av,vol,omegaA,omegaC,pH

            if process_top100:
                #----
                # top100m
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[ind1,:]
                omegaC = omegaC[ind1,:]
                pH = pH[ind1,:]
                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                #ind_av = omegaA.mask==False
                ind_av = np.where(omegaA>0)[0]
                vv = volume_2d[ind1,:]
                vol = vv[:,ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_top100_1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_top100_1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_top100_1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_top100_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_top100_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_top100_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_top100_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_top100_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low


                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_top100_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_top100_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_top100_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_top100_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_top100_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                del ind_av,vol,omegaA,omegaC,pH

                
            if process_top200:
                #----
                # top200m
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[ind10,:]
                omegaC = omegaC[ind10,:]
                pH = pH[ind10,:]
                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                #ind_av = omegaA.mask==False
                ind_av = np.where(omegaA>0)[0]
                vv = volume_2d[ind10,:]
                vol = vv[:,ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_top200_1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_top200_1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_top200_1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_top200_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_top200_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_top200_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_top200_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_top200_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low


                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_top200_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_top200_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_top200_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_top200_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_top200_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                del ind_av,vol,omegaA,omegaC,pH
                
                
            if process_100_500:
                #----
                # 100-500m
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[ind2,:]
                omegaC = omegaC[ind2,:]
                pH = pH[ind2,:]
                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                ind_av = omegaA.mask==False
                vv = volume_2d[ind2,:]
                vol = vv[:,ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_100_500_1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_100_500_1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_100_500_1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_100_500_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_100_500_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_100_500_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_100_500_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_100_500_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low


                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_100_500_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_100_500_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_100_500_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_100_500_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_100_500_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                del ind_av,vol,omegaA,omegaC,pH

            if process_0_10:
                #----
                # 0-10m
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[ind7,:]
                omegaC = omegaC[ind7,:]
                pH = pH[ind7,:]
                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                ind_av = omegaA.mask==False
                vv = volume_2d[ind7,:]
                vol = vv[:,ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_0_10_1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_0_10_1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_0_10_1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_0_10_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_0_10_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_0_10_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_0_10_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_0_10_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_0_10_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_0_10_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_0_10_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_0_10_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_0_10_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                del ind_av,vol,omegaA,omegaC,pH

            if process_10_100:
                #----
                # 10-100m
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[ind8,:]
                omegaC = omegaC[ind8,:]
                pH = pH[ind8,:]
                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                ind_av = omegaA.mask==False
                vv = volume_2d[ind8,:]
                vol = vv[:,ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_10_100_1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_10_100_1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_10_100_1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_10_100_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_10_100_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_10_100_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_10_100_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_10_100_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_10_100_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_10_100_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_10_100_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_10_100_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_10_100_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low


                del ind_av,vol,omegaA,omegaC,pH


            if process_below500:
                #----
                # below 500m
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[ind3,:]
                omegaC = omegaC[ind3,:]
                pH = pH[ind3,:]
                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                ind_av = omegaA.mask==False
                vv = volume_2d[ind3,:]
                vol = vv[:,ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_below500_1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_below500_1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_below500_1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_below500_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_below500_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_below500_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_below500_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_below500_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low


                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_below500_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_below500_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_below500_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_below500_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_below500_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low


                del ind_av,vol,omegaA,omegaC,pH

            if process_above2000:
                #----
                # above 2000m
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[ind9,:]
                omegaC = omegaC[ind9,:]
                pH = pH[ind9,:]
                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                ind_av = omegaA.mask==False
                vv = volume_2d[ind9,:]
                vol = vv[:,ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_above2000_1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_above2000_1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_above2000_1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_above2000_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_above2000_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_above2000_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_above2000_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_above2000_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low


                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_above2000_1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_above2000_1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_above2000_1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_above2000_1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_above2000_1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low


                del ind_av,vol,omegaA,omegaC,pH

            if process_water_column:
                #----
                # whole water column
                #----
                omegaA = f1.variables['OmegaA'][mm,:,:]
                omegaC = f2.variables['OmegaC'][mm,:,:]
                pH     = f3.variables['pH'][mm,:,:]

                omegaA = omegaA[:,ind_regions].ravel()
                omegaC = omegaC[:,ind_regions].ravel()
                pH = pH[:,ind_regions].ravel()

                # get avg over whole shelf area
                ind_av = omegaA.mask==False
                vol = volume_2d[:,ind_regions].ravel()[ind_av]
                omegaA_avg_shelf1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                del ind_av,vol,omegaA,omegaC,pH

            if process_surface:
                #----
                # surface only
                #----
                omegaA = f1.variables['OmegaA'][mm,0,:]
                omegaC = f2.variables['OmegaC'][mm,0,:]
                pH     = f3.variables['pH'][mm,0,:]

                omegaA = omegaA[ind_regions].ravel()
                omegaC = omegaC[ind_regions].ravel()
                pH = pH[ind_regions].ravel()

                # get avg over whole shelf area
                ind_av = omegaA.mask==False
                vv = volume_2d[0,:]
                vol = vv[ind_regions].ravel()[ind_av]
                del vv
                omegaA_avg_shelf_srf1[mm,yy,rr] = np.sum(omegaA[ind_av]*vol/np.sum(vol))
                omegaC_avg_shelf_srf1[mm,yy,rr] = np.sum(omegaC[ind_av]*vol/np.sum(vol))
                pH_avg_shelf_srf1[mm,yy,rr]     = np.sum(pH[ind_av]*vol/np.sum(vol))

                # get volume of undersaturated waters (omegaA & omegaC)
                ind_low = np.where(omegaA[ind_av]<1)[0]
                omegaA_vol_underst_srf1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<0.5)[0]
                omegaA_vol_underst_srf1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]<1.5)[0]
                omegaA_vol_underst_srf1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=1.5)[0]
                omegaA_vol_underst_srf1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaA[ind_av]>=2)[0]
                omegaA_vol_underst_srf1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                ind_low = np.where(omegaC[ind_av]<1)[0]
                omegaC_vol_underst_srf1[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<0.5)[0]
                omegaC_vol_underst_srf1_05[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]<1.5)[0]
                omegaC_vol_underst_srf1_15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=1.5)[0]
                omegaC_vol_underst_srf1_gt15[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low
                ind_low = np.where(omegaC[ind_av]>=2)[0]
                omegaC_vol_underst_srf1_gt2[mm,yy,rr] = 100*np.sum(vol[ind_low])/np.sum(vol)
                del ind_low

                del ind_av,vol,omegaA,omegaC,pH
            
            
    #-----
    # save as netcdf file
    #-----
    
    save_netcdf = True
    # only save every year (takes too much time otherwise)
    if save_netcdf:
        if save_netcdf:
            if save_netcdf:
                savepath = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/regional/'
                netcdf_name = 'Carbonate_chemistry_regional_'+which_sim+'_'+str(year_list1[0])+'_'+str(year_list1[-1])+'_v3.nc'

                if not os.path.exists(savepath+netcdf_name):
                    print('Create file '+savepath+netcdf_name)
                    w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
                    w_nc_fid.regions1 = '1=all shelves, 2=Weddell Sea, 3=eWeddell Sea, 4=Prydz, 5=Adelie,'+\
                                ' 6=Ross Sea, 7=Amundsen/Bellingshausen, 8=WAP'
                    w_nc_fid.regions2 = '9=RossSea_MPA, 10=RossSea_MPA_shelf, 11=RossSea_MPA_openOcean'
                    w_nc_fid.regions3 = '12=WeddellSea_MPA, 13=WeddellSea_MPA_shelf, 14=WeddellSea_MPA_openOcean'
                    w_nc_fid.regions4 = '15=Orkney_MPA, 16=AntarcticPeninsula_MPA'
                    w_nc_fid.regions5 = '17=East_Antarctica_1_MPA, 18=East_Antarctica_2_MPA, 19=East_Antarctica_3_MPA, 20=East_Antarctica_all_MPA'
                    w_nc_fid.regions6 = '21=East_Antarctica_1_MPA_shelf, 22=East_Antarctica_2_MPA_shelf,'+\
                                ' 23=East_Antarctica_3_MPA_shelf, 24=East_Antarctica_all_MPA_shelf'
                    w_nc_fid.regions7 = '25=East_Antarctica_1_MPA_openOcean, 26=East_Antarctica_2_MPA_openOcean,'+\
                                ' 27=East_Antarctica_3_MPA_openOcean, 28=East_Antarctica_all_MPA_openOcean'
                       
                    w_nc_fid.mocsy_source = 'https://github.com/jamesorr/mocsy'
                    w_nc_fid.mocsy_dir    = '/home/ollie/ncara/scripts/mocsy/mocsy'
                    w_nc_fid.script    = '/global/homes/c/cnissen/scripts/plot_PAPER_carbonate_chemistry_regional_save_netcdf_v3.ipynb'
                    # create dimension & variable
                    w_nc_fid.createDimension('month', 12)  # monthly
                    w_nc_fid.createDimension('year', len(year_list1)) 
                    w_nc_fid.createDimension('subregions', len(subregions)) 
                    w_nc_fid.createDimension('nodes_2d', len(topo)) 
                    w_nc_fid.createVariable('year', 'f4',('year'))
                    w_nc_fid.variables['year'][:] = year_list1

                    w_nc_fid.createVariable('mask_subregions', 'f4',('nodes_2d'))
                    w_nc_fid.variables['mask_subregions'][:] = shelf_ind

                    for vv in range(0,len(vari_list)):
                        w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('month','year','subregions'))
                    w_nc_fid.close()
                else: # file exists, but check if variables exist as well
                    w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
                    for vv in range(0,len(vari_list)):
                        try:
                            w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('month','year','subregions'))
                        except: 
                            if yy==0:
                                print (vari_list[vv]+' exists in file')
                            pass
                    w_nc_fid.close()
                    

                for vv in range(0,len(vari_list)):
                    w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
                    if vari_list[vv] in ['omegaA_avg_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_srf1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_top10_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_top20_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_top50_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_top100_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_top200_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_100_500_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_below500_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_above2000_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_0_10_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_avg_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_avg_shelf_10_100_1[:,yy,:]
                    #########
                    elif vari_list[vv] in ['omegaC_avg_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_srf1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_top10_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_top20_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_top50_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_top100_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_top200_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_100_500_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_below500_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_above2000_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_0_10_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_avg_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_avg_shelf_10_100_1[:,yy,:]
                    #########
                    elif vari_list[vv] in ['pH_avg_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_srf1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_top10_1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_top20_1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_top50_1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_top100_1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_top200_1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_100_500_1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_below500_1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_above2000_1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_0_10_1[:,yy,:]
                    elif vari_list[vv] in ['pH_avg_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = pH_avg_shelf_10_100_1[:,yy,:]
                    #########
                    elif vari_list[vv] in ['omegaA_vol_below1_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_srf1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top10_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top20_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top50_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top100_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top200_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_100_500_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_below500_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_above2000_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_0_10_1[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below1_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_10_100_1[:,yy,:]
                    #########
                    elif vari_list[vv] in ['omegaC_vol_below1_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_srf1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top10_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top20_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top50_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top100_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top200_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_100_500_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_below500_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_above2000_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_0_10_1[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below1_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_10_100_1[:,yy,:]
                    #########
                    #########
                    elif vari_list[vv] in ['omegaA_vol_below05_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_srf1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top10_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top20_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top50_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top100_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top200_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_100_500_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_below500_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_above2000_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_0_10_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below05_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_10_100_1_05[:,yy,:]
                    #########
                    elif vari_list[vv] in ['omegaC_vol_below05_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_srf1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top10_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top20_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top50_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top100_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top200_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_100_500_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_below500_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_above2000_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_0_10_1_05[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below05_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_10_100_1_05[:,yy,:]
                    #########
                    elif vari_list[vv] in ['omegaA_vol_below15_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_srf1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top10_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top20_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top50_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top100_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top200_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_100_500_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_below500_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_above2000_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_0_10_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_below15_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_10_100_1_15[:,yy,:]
                    #########
                    elif vari_list[vv] in ['omegaC_vol_below15_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_srf1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top10_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top20_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top50_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top100_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top200_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_100_500_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_below500_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_above2000_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_0_10_1_15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_below15_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_10_100_1_15[:,yy,:]
                    #########
                    elif vari_list[vv] in ['omegaA_vol_above15_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_srf1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top10_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top20_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top50_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top100_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top200_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_100_500_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_below500_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_above2000_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_0_10_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above15_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_10_100_1_gt15[:,yy,:]
                    #########
                    elif vari_list[vv] in ['omegaC_vol_above15_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_srf1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top10_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top20_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top50_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top100_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top200_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_100_500_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_below500_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_above2000_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_0_10_1_gt15[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above15_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_10_100_1_gt15[:,yy,:]
                    #########
                    elif vari_list[vv] in ['omegaA_vol_above2_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_srf1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top10_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top20_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top50_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top100_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_top200_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_100_500_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_below500_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_above2000_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_0_10_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaA_vol_above2_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaA_vol_underst_10_100_1_gt2[:,yy,:]
                    #########
                    elif vari_list[vv] in ['omegaC_vol_above2_srf']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_srf1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_top10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top10_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_top20']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top20_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_top50']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top50_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_top100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top100_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_top200']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_top200_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_100_500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_100_500_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_below500']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_below500_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_above2000']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_above2000_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_water_column']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_0_10']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_0_10_1_gt2[:,yy,:]
                    elif vari_list[vv] in ['omegaC_vol_above2_10_100']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = omegaC_vol_underst_10_100_1_gt2[:,yy,:]

                    w_nc_fid.close()  
                del netcdf_name

                            

    # close netcdf files
    f1.close()
    f2.close()
    f3.close()

print ('done')


[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031
 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045
 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059
 2060 2061 2062 2063 2064 2065 2066 2067 2068 2069 2070 2071 2072 2073
 2074 2075 2076 2077 2078 2079 2080 2081 2082 2083 2084 2085 2086 2087
 2088 2089 2090 2091 2092 2093 2094 2095 2096 2097 2098 2099 2100]
year 1990...


100%|██████████| 12/12 [00:16<00:00,  1.36s/it]


omegaA_avg_top200 exists in file
omegaC_avg_top200 exists in file
pH_avg_top200 exists in file
omegaA_vol_below1_top200 exists in file
omegaC_vol_below1_top200 exists in file
omegaA_vol_below05_top200 exists in file
omegaC_vol_below05_top200 exists in file
omegaA_vol_below15_top200 exists in file
omegaC_vol_below15_top200 exists in file
omegaA_vol_above15_top200 exists in file
omegaC_vol_above15_top200 exists in file
omegaA_vol_above2_top200 exists in file
omegaC_vol_above2_top200 exists in file
year 1991...


100%|██████████| 12/12 [00:18<00:00,  1.54s/it]


year 1992...


...

year 2016...


100%|██████████| 12/12 [00:14<00:00,  1.24s/it]


year 2017...


100%|██████████| 12/12 [00:14<00:00,  1.25s/it]


done
